In [1]:
import os
import ot
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import argparse
import time
from scipy.spatial import distance
from sklearn.cluster import KMeans



In [2]:
seed = 2000

np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [3]:


class NumpyDataset:
    def __init__(self, data, colors):
        self.data = data
        self.colors = colors

    def __getitem__(self, index):
        return self.data[index], self.colors[index]

    def __len__(self):
        return len(self.data)


class NumpyDataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, drop_last=False, output_ids=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        self.output_ids = output_ids

    def __iter__(self):
        self.indices = np.arange(len(self.dataset))
        if self.shuffle:
            np.random.shuffle(self.indices)
        if self.drop_last:
            self.indices = self.indices[:len(self.indices) - len(self.indices) % self.batch_size]
        return self

    def __next__(self):
        if len(self.indices) == 0:
            raise StopIteration
        
        indices = self.indices[:self.batch_size]
        self.indices = self.indices[self.batch_size:]
        
        if self.output_ids:
            batch = []
            for i in indices:
                batch.append((i, self.dataset[i][0], self.dataset[i][1]))
            ids, data, labels = zip(*batch)
            return np.array(ids), np.array(data), np.array(labels)
        else:
            batch = [self.dataset[i] for i in indices]
            data, labels = zip(*batch)
            return np.array(data), np.array(labels)


def optimize_coupling(xs, centers, numItermax=5000, numThreads=3):

    # only for binary sensitive attribute
    n_colors = 2
    assert n_colors == len(xs)
    ns = [len(xs_i) for xs_i in xs]
    pis = np.array(ns) / np.sum(ns)
    pi_0, pi_1 = pis[0], pis[1]

    n_0, n_1 = ns[0], ns[1]
    w_0, w_1 = np.ones(n_0) / n_0, np.ones(n_1) / n_1
    
    D_01 = 2 * pi_0 * pi_1 * ot.dist(xs[0], xs[1])
    
    Taxs_0_1 = pi_0 * np.repeat(xs[0], n_1, 0) + pi_1 * np.tile(xs[1], (n_0, 1))
    mid_distances = distance.cdist(Taxs_0_1, centers, metric='minkowski', p=2)
    mid_assignments = np.argmin(mid_distances, axis=1)
    C_01 = (mid_distances[np.arange(mid_distances.shape[0]), mid_assignments]**2)
    # C_mami = C_mami.reshape(n_1, n_0).T
    C_01 = C_01.reshape(n_0, n_1)
    
    Coupling_0_1 = ot.emd(w_0, w_1, D_01+C_01, numItermax=numItermax, numThreads=numThreads)

    return Taxs_0_1, Coupling_0_1.flatten(), n_0, n_1



def optimize_center(Taxs, gammas, centers, centers_module, centers_optimizer, K, seed=2024, gradient_descent=False, use_cuda=False):
    if gradient_descent:
        Taxs = torch.from_numpy(Taxs).float().cuda() if use_cuda else torch.from_numpy(Taxs).float()
        gammas = torch.from_numpy(gammas).float().cuda() if use_cuda else torch.from_numpy(gammas).float()
        for _ in range(20):
            distances = torch.cdist(Taxs, centers_module.weight, p=2)
            assignments = torch.argmin(distances, dim=1)
            energy = (gammas * (distances[torch.arange(distances.shape[0]), assignments]**2)).sum()
            centers_optimizer.zero_grad()
            energy.backward()
            centers_optimizer.step()
        new_centers = centers_module.weight.data.cpu().detach().numpy()
    else:
        kmeans = KMeans(n_clusters=K, init=centers, random_state=seed)
        # kmeans = KMeans(n_clusters=K, random_state=seed)
        kmeans.fit(X=Taxs, sample_weight=gammas)
        new_centers = kmeans.cluster_centers_
    return new_centers



def random_hard_assigning(arr):
    max_values = np.max(arr, axis=1)
    max_mask = arr == max_values[:, None]
    chosen_indices = np.array([np.random.choice(np.where(row)[0]) for row in max_mask])
    return chosen_indices



def assigning(xs, Taxs, gammas, centers, K):
    """
    xs: (n, d) # n = n0 + n1
    colors: (n, )
    Taxs: (n0 * n1, d)
    gammas: (n0 * n1, )
    n_majors: (B, ) # B = batch_size
    n_minors: (B, ) # B = batch_size
    centers: (K, d)
    K: int
    """
    
    color_xs = [[], []]
    color_assignments = [[], []]
    for sub_xs, sub_Taxs, sub_gammas in zip(xs, Taxs, gammas):
        
        n_0, n_1 = sub_xs[0].shape[0], sub_xs[1].shape[0]
        
        sub_distances = distance.cdist(sub_Taxs, centers, metric='minkowski', p=2)
        sub_assignments = np.argmin(sub_distances, axis=1)
        
        # shape: (n0, n1)
        sub_gammas_i_j = sub_gammas.reshape(n_0, n_1)
        sub_assignments_i_j = sub_assignments.reshape(n_0, n_1)

        # for s = 0
        prob_assignments_0 = np.zeros(shape=(n_0, K))
        for row in range(n_0):
            for k in range(K):
                prob_assignments_0[row, k] = n_0 * np.sum(sub_gammas_i_j[row][sub_assignments_i_j[row] == k])

        # for s = 1
        prob_assignments_1 = np.zeros(shape=(n_1, K))
        for col in range(n_1):
            for k in range(K):
                prob_assignments_1[col, k] = n_1 * np.sum(sub_gammas_i_j.T[col][sub_assignments_i_j.T[col] == k])

        color_xs[0].append(sub_xs[0])
        color_xs[1].append(sub_xs[1])
        color_assignments[0].append(random_hard_assigning(prob_assignments_0))
        color_assignments[1].append(random_hard_assigning(prob_assignments_1))
    
    color_xs[0] = np.concatenate(color_xs[0])
    color_xs[1] = np.concatenate(color_xs[1])
    color_assignments[0] = np.concatenate(color_assignments[0])
    color_assignments[1] = np.concatenate(color_assignments[1])

    return color_xs, color_assignments



def evaluation(color_xs, color_assignments, centers, n_color, K):
    
    # objectives
    cluster_cnts = []
    objective = 0.0
    for xs_i, assignments_i in zip(color_xs, color_assignments):
        distances_i = distance.cdist(xs_i, centers, metric='minkowski', p=2)
        objective += (distances_i[np.arange(distances_i.shape[0]), assignments_i]**2).sum()
        
        sub_cluster_cnts = []
        for k in range(K):
            sub_cluster_cnts.append((assignments_i == k).sum())
        cluster_cnts.append(sub_cluster_cnts)
    
    # balance
    all_cluster_cnts = np.array(cluster_cnts).sum(axis=0)
    k_ratio = all_cluster_cnts / all_cluster_cnts.sum()
    s_balances = []
    for color in range(n_color):
        s_balances.append((np.array(cluster_cnts[color]) / np.sum(cluster_cnts[color])) / k_ratio)
    balance = np.array(s_balances).min(axis=1).min()

    return objective, balance



In [4]:
seed= 2024
full_batch=True
batch_size= 4096

gradient_descent = True
use_cuda = True

iters = 10
numItermax = 1000

l2_normalize = True

In [5]:
data_path ="data_combined_new.csv"
data = pd.read_csv(data_path)

In [6]:
data

,user_id,movie_id,movie_title,sensitive,age,occupation,zip_code,genre_Action,genre_Adventure,genre_Animation,...,genre_rating_Fantasy,genre_rating_Film-Noir,genre_rating_Horror,genre_rating_Musical,genre_rating_Mystery,genre_rating_Romance,genre_rating_Sci-Fi,genre_rating_Thriller,genre_rating_War,genre_rating_Western
0,1,53,One Flew Over the Cuckoo's Nest (1975) James a...,1,1,10,48067,5,5,18,...,4.00,NaN,NaN,4.285714,NaN,3.666667,4.333333,3.666667,5.000000,NaN
1,2,129,"Shine (1996) Verdict, The (1982) Shall We Danc...",0,56,16,70072,56,19,0,...,3.00,4.000000,3.000000,NaN,3.333333,3.708333,3.588235,3.483871,3.733333,4.333333
2,3,51,"Animal House (1978) Full Monty, The (1997) Mis...",0,25,15,55117,23,25,3,...,4.50,NaN,2.666667,4.000000,3.000000,3.800000,3.833333,3.800000,4.000000,4.666667
3,4,21,"Hustler, The (1961) Star Wars: Episode VI - Re...",0,45,7,02460,19,6,0,...,4.50,NaN,4.333333,NaN,NaN,4.000000,3.555556,3.500000,3.333333,4.500000
4,5,198,Who Framed Roger Rabbit? (1988) Gods and Monst...,0,25,20,55455,31,9,4,...,NaN,4.000000,2.800000,3.333333,3.125000,3.100000,3.066667,2.846154,3.500000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,888,"Wedding Gift, The (1994) Spanking the Monkey (...",1,25,15,32603,154,83,34,...,3.00,4.058824,2.986486,3.709677,3.411765,3.352459,2.834320,3.142857,3.785714,3.642857
6036,6037,202,"Terminator 2: Judgment Day (1991) Insider, The...",1,45,1,76006,28,9,1,...,4.25,3.444444,4.111111,4.000000,3.692308,3.681818,3.692308,3.705882,4.000000,3.750000
6037,6038,20,Walkabout (1971) Gone with the Wind (1939) Har...,1,56,1,14706,2,1,3,...,NaN,NaN,2.500000,NaN,NaN,4.166667,4.000000,NaN,4.000000,NaN
6038,6039,123,Aladdin (1992) Doctor Zhivago (1965) Evita (19...,1,45,0,01060,8,10,13,...,3.60,4.500000,4.000000,3.690476,4.176471,3.800000,4.250000,4.142857,4.111111,4.500000


In [7]:
# 장르별 개수, rating 컬럼만 잘라서 사용
new_data = data.iloc[:,7:].fillna(0)
#new_data = new_data.values / new_data.values.sum(axis=1).reshape(-1,1)
new_data_gender = data["sensitive"].values

In [8]:
# 데이터 및 하이퍼파라미터 지정
np_data = np.array(new_data)
np_colors = np.array(new_data_gender)
K = 5
d = np_data.shape[1]
n_color = 2

In [9]:
# sensitive attribute에 따른 데이터 개수 확인
xs = []
for color in range(n_color):
    xs.append(np_data[np_colors == color])
    print(f'[Info] Data shape for {color}th color: {np_data[np_colors == color].shape}')
colors = [i*np.ones(xs_i.shape[0]) for i, xs_i in enumerate(xs)]

[Info] Data shape for 0th color: (4331, 36)
[Info] Data shape for 1th color: (1709, 36)


In [10]:
# 데이터세트 데이터로더 
dset = NumpyDataset(np.concatenate(xs), np.concatenate(colors))
batch_size = batch_size
if full_batch:
    batch_size = len(np.concatenate(xs))
dloader = NumpyDataLoader(dset, batch_size=batch_size,
                            shuffle=True, drop_last=False)

In [11]:
# unfair clustering!
# 그냥 k-means clustering을 적용했을 때의 결과. balance가 1에 가까울 수록 fair clustering이라고 볼 수 있음.
kmeans = KMeans(n_clusters=K, random_state=2024)
kmeans.fit(X=np.concatenate(xs))
unfair_assignments = kmeans.predict(X=np.concatenate(xs))
unfair_assignments = [unfair_assignments[:xs[0].shape[0]], unfair_assignments[xs[0].shape[0]:]]
unfair_centers = kmeans.cluster_centers_

objective, balance = evaluation(xs, unfair_assignments, unfair_centers, n_color, K)
print(f'[Unfair] Energy / Balance: {objective:.3f} / {balance:.3f}')

[Unfair] Energy / Balance: 31468348.052 / 0.342


In [12]:
# initial centers 지정
centers = np.random.normal(0, 1, (K, d))
centers_module = None
centers_optimizer = None
if gradient_descent:
    lr = 5e-3 
    centers_module = nn.Linear(d, K, bias=False)
    centers_optimizer = torch.optim.Adam(centers_module.parameters(), lr=lr)
    if use_cuda:
        centers_module = centers_module.cuda()

In [13]:
# optimizing 변수들 초기화
elapsed_times = []
best_it, best_original_energy, best_energy, best_balance = 0, 1e+10, 1e+10, 0.0
subbest_it, subbest_original_energy, subbest_energy, subbest_balance = 0, 1e+10, 1e+10, 0.0


In [14]:
for it in range(iters):
    start_time = time.time()
    it += 1
    if gradient_descent:
        centers = centers_module.weight.data.cpu().detach().numpy()
    
    all_xs, all_Taxs, all_gammas, all_colors = [], [], [], []
    all_n_0, all_n_1 = [], []
    for batch_xs, batch_colors in dloader:
        sub_batch_xs = []
        for color in range(n_color):
            sub_batch_xs.append(batch_xs[batch_colors == color])
        sub_batch_colors = [i*np.ones(sub_batch_xs_i.shape[0]) for i, sub_batch_xs_i in enumerate(sub_batch_xs)]
        # optimize coupling : 배치 내에서 남-여 1:1 조합의 확률을 부여하고 중점을 구함.
        # sub_Tax_i_j - 중점  sub_gamma_i_j - 확률
        sub_Tax_i_j, sub_gamma_i_j, sub_n_0, sub_n_1 = optimize_coupling(sub_batch_xs, centers, numItermax=numItermax)
        
        all_xs.append(sub_batch_xs)
        all_Taxs.append(sub_Tax_i_j)
        all_gammas.append(sub_gamma_i_j)
        all_colors.append(np.concatenate(sub_batch_colors))
        all_n_0.append(sub_n_0)
        all_n_1.append(sub_n_1)

    all_colors = np.concatenate(all_colors)

    # finding center
    flat_all_Taxs = np.concatenate(all_Taxs)
    flat_all_gammas = np.concatenate([sub_gamma / sub_gamma.shape[0] for sub_gamma in all_gammas])
    # 중점을 점으로, 확률을 weight로 하는 weighted k-means 각 클러스터의 center를 구함.
    centers = optimize_center(flat_all_Taxs, flat_all_gammas,
                                centers, centers_module, centers_optimizer, K, seed=seed,
                                gradient_descent=gradient_descent, use_cuda=use_cuda)

    elapsed_times.append(time.time() - start_time)

    # assigning: 확률로 soft하게 구한 값을 hard하게 샘플링.
    color_xs, color_assignments = assigning(all_xs, all_Taxs, all_gammas, centers, K)
    objective, balance = evaluation(color_xs, color_assignments, centers, n_color, K)

    print(f'[{it}/{iters}] Energy / Balance: {objective:.3f} / {balance:.3f}')
    
    if balance > best_balance:
        best_it = it
        best_balance = balance
        best_energy = objective
        best_original_energy = objective
    if objective < subbest_energy:
        subbest_it = it
        subbest_balance = balance
        subbest_energy = objective
        subbest_original_energy = objective
        
elapsed_time_per_iter = np.mean(elapsed_times)
elapsed_time_total = np.sum(elapsed_times)

# results
results = {'type': [type],
            'seed':[seed],
            'gradient_descent': [gradient_descent],
            'iters': [iters],
            'full_batch': [f'real_{full_batch}'],
            'batch_size': [batch_size],
            'l2_normalize': [l2_normalize],
            'elapsed_time_per_iter': [elapsed_time_per_iter],
            'elapsed_time_total': [elapsed_time_total],
            'best_it': [best_it],
            'best_original_energy': [best_original_energy],
            'best_energy': [best_energy],
            'best_balance': [best_balance],
            'subbest_it': [subbest_it],
            'subbest_original_energy': [subbest_original_energy],
            'subbest_energy': [subbest_energy],
            'subbest_balance': [subbest_balance]
            }

columns = list(results.keys())
df_results = pd.DataFrame(results, columns=columns)
#result_name = f'results/FCA/{args.data_name}_FCA.csv'

print(f'[BEST balance/{iters}] Energy / Balance: {best_energy:.3f} / {best_balance:.3f}')
print(f'[BEST energy/{iters}] Energy / Balance: {subbest_energy:.3f} / {subbest_balance:.3f}')



/usr/local/miniconda3/envs/nine/lib/python3.9/site-packages/ot/lp/__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


[1/10] Energy / Balance: 194789814.615 / 0.863
[2/10] Energy / Balance: 194477297.084 / 0.818
[3/10] Energy / Balance: 194183185.077 / 0.842
[4/10] Energy / Balance: 193903600.515 / 0.819
[5/10] Energy / Balance: 193639377.123 / 0.862
[6/10] Energy / Balance: 193379230.996 / 0.822
[7/10] Energy / Balance: 193103693.668 / 0.898
[8/10] Energy / Balance: 192842369.770 / 0.860
[9/10] Energy / Balance: 192593330.713 / 0.778
[10/10] Energy / Balance: 192303615.249 / 0.761
[BEST balance/10] Energy / Balance: 193103693.668 / 0.898
[BEST energy/10] Energy / Balance: 192303615.249 / 0.761


In [21]:
# Grouping
score = 0.0
num_data = 0.0
# 5개의 cluster가 만들어졌으므로 각 cluster내부에서 영화 rating의 1,2 위를 추천.
for group_num in [0,1,2,3,4]:

    group_man_bool = color_assignments[0] == group_num
    group_woman_bool = color_assignments[1] == group_num
    group_man = color_xs[0][group_man_bool,:]
    group_woman = color_xs[1][group_woman_bool,:]

    group_all = np.concatenate([group_man,group_woman])

    # 추천할 장르 고르기
    recommend_genre = pd.DataFrame( group_all[:,18:].argmax(axis=1) ).value_counts()

    # 각각 1,2,3 등 장르
    recommend_genre_0 = recommend_genre.index[0][0]
    recommend_genre_1 = recommend_genre.index[1][0]
    recommend_genre_2 = recommend_genre.index[2][0]

    #score = 0.0
    # 1,2 위 장르를 추천했을 때, 개인이 해당 장르에 대한 rating이 3.0 이상이면 추천에 성공(Y_hat = Y = 1)
    score += np.sum( (group_all[:,18:][:,recommend_genre_0] > 3.0) | (group_all[:,18:][:,recommend_genre_1] > 3.0) )
    num_data += group_all.shape[0]
print(score/num_data)

0.8735099337748344


In [22]:
## 위 같은 추천의 정확도를 각 그룹의 성별 별로 구하고 차이가 얼마나 되는지 알아보자.
for group_num in [0,1,2,3,4]:
    group_man_bool = color_assignments[0] == group_num
    group_woman_bool = color_assignments[1] == group_num
    group_man = color_xs[0][group_man_bool,:]
    group_woman = color_xs[1][group_woman_bool,:]

    group_all = np.concatenate([group_man,group_woman])

    recommend_genre = pd.DataFrame( group_all[:,18:].argmax(axis=1) ).value_counts()

    recommend_genre_0 = recommend_genre.index[0][0]
    recommend_genre_1 = recommend_genre.index[1][0]
    recommend_genre_2 = recommend_genre.index[2][0]
    
    rating_posivie = 3.0
    
    man_score = 0.0
    man_score += np.sum( (group_man[:,18:][:,recommend_genre_0] > rating_posivie) | (group_man[:,18:][:,recommend_genre_1] > rating_posivie) )
    
    
    woman_score = 0.0
    woman_score += np.sum( (group_woman[:,18:][:,recommend_genre_0] > rating_posivie) | (group_woman[:,18:][:,recommend_genre_1] > rating_posivie) )
    
    print(f"Man :{man_score/group_man.shape[0]} , Woman : {woman_score/group_woman.shape[0]}")

Man :0.9425531914893617 , Woman : 0.9302325581395349
Man :0.9331941544885177 , Woman : 0.9012345679012346
Man :0.8076152304609219 , Woman : 0.8662790697674418
Man :0.6531049250535332 , Woman : 0.6303030303030303
Man :0.9184602649006622 , Woman : 0.8695652173913043


In [23]:
## 성별 별로
man_score = 0.0
woman_score = 0.0
man_num = 0.0
woman_num =0.0
## 위 같은 추천의 정확도를 모든 그룹에서 평균냈을 때 성별 별로 차이가 나는지 확인하자.
for group_num in [0,1,2,3,4]:
    group_man_bool = color_assignments[0] == group_num
    group_woman_bool = color_assignments[1] == group_num
    group_man = color_xs[0][group_man_bool,:]
    group_woman = color_xs[1][group_woman_bool,:]

    group_all = np.concatenate([group_man,group_woman])

    recommend_genre = pd.DataFrame( group_all[:,18:].argmax(axis=1) ).value_counts()

    recommend_genre_0 = recommend_genre.index[0][0]
    recommend_genre_1 = recommend_genre.index[1][0]
    recommend_genre_2 = recommend_genre.index[2][0]
    
    rating_posivie = 3.00

    man_score += np.sum( (group_man[:,18:][:,recommend_genre_0] >= rating_posivie) | (group_man[:,18:][:,recommend_genre_1] >= rating_posivie) )
    
    woman_score += np.sum( (group_woman[:,18:][:,recommend_genre_0] >= rating_posivie) | (group_woman[:,18:][:,recommend_genre_1] >= rating_posivie) )
    
    man_num += group_man.shape[0]
    woman_num += group_woman.shape[0]

# fair r기준 dp
print(f"Man :{man_score/man_num} , Woman : {woman_score/woman_num} , Fair : {np.abs( man_score/man_num- woman_score/woman_num ) } ")

Man :0.9131840221657815 , Woman : 0.9040374488004681 , Fair : 0.009146573365313437 
